In [28]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import Holt # Exponential Smoothing
from sklearn.metrics import mean_squared_error
from math import sqrt
import warnings
warnings.filterwarnings("ignore")

In [15]:
# 1. Data Preparation

# Load the dataset and parse the 'date' column as the index
df = pd.read_csv(
    '/content/exchange_rate (2).csv',
    index_col='date',
    parse_dates=True,
    dayfirst=True # Ensure correct DD-MM-YYYY format interpretation
)

In [16]:
# Rename the exchange rate column for simplicity
df.columns = ['Rate']


In [17]:
# Define the split point: last 30 observations for testing
test_size = 30
train = df[:-test_size]
test = df[-test_size:]


In [18]:
# 2. ARIMA Model Implementation

# Define ARIMA order (p, d, q).
# We use (5, 1, 0): 5 AR terms, 1st difference for stationarity, 0 MA terms.
arima_order = (5, 1, 0)


In [19]:
# Fit the ARIMA model on the training data
# Note: The data is passed directly; no explicit loop is needed.
arima_model_fit = ARIMA(train['Rate'], order=arima_order).fit()


In [20]:
# Forecast the next 30 steps (the size of the test set)
arima_forecast = arima_model_fit.forecast(steps=test_size)


In [21]:
# Calculate RMSE for ARIMA
arima_rmse = sqrt(mean_squared_error(test['Rate'], arima_forecast))


In [22]:
# 3. Exponential Smoothing Model Implementation (Holt's Linear Trend)

# Fit the Holt model with an additive trend (best for non-seasonal, trending data)
holt_model_fit = Holt(train['Rate'], initialization_method='estimated').fit()


In [23]:
# Forecast the next 30 steps
holt_forecast = holt_model_fit.forecast(steps=test_size)


In [24]:
# Calculate RMSE for Holt
holt_rmse = sqrt(mean_squared_error(test['Rate'], holt_forecast))


In [25]:
#  4. Final Results (Stored in variables)

# Store the final Root Mean Squared Error (RMSE) values for comparison
arima_model_rmse = arima_rmse
holt_model_rmse = holt_rmse

In [30]:
arima_model_rmse

0.016651576590466198

In [31]:
holt_model_rmse

0.01649594238711105

 ## Time Series Forecasting Model Comparison

The analysis compared two models for forecasting the exchange rate over the last 30 observations: **ARIMA(5, 1, 0)** and **Holt's Linear Trend (Exponential Smoothing)**. The performance metric used is the **Root Mean Squared Error (RMSE)**.

| Model | Root Mean Squared Error (RMSE) |
| :--- | :--- |
| **ARIMA(5, 1, 0)** | $0.0167$ |
| **Holt's Linear Trend** | $\mathbf{0.0165}$ |

-----

## Conclusion

The objective was to identify the best model for short-term exchange rate forecasting.

1.  **Model Performance:** The **Holt's Linear Trend** model yielded the lowest RMSE of **$0.0165$**, indicating slightly superior predictive accuracy compared to the ARIMA model's RMSE of $0.0167$.
2.  **Model Interpretation:**
      * **Holt's Linear Trend** is well-suited for this dataset because the exchange rate exhibits a **clear trend** but likely **lacks a strong, predictable seasonality**. The model's strength lies in its ability to smoothly adapt to this non-seasonal trend.
      * **ARIMA** models are more complex, requiring specific differencing ($d=1$) and parameter choices ($p=5, q=0$). While its performance was close, the simpler Holt's method proved marginally more effective in capturing the underlying dynamics for this specific forecasting horizon.

**Final Choice:** **Holt's Linear Trend** is the preferred model for short-term forecasting on this exchange rate dataset.